# Tutorial : Spectral early warning signals with *ewstools*

By the end of this tutorial you should know how to:
- Compute spectral early warning signals as in [Bury et al. (2021)](https://royalsocietypublishing.org/doi/full/10.1098/rsif.2020.0482)

Total run time : 2 min 18 s on Macbook Air (M1, 2020)
